## Step 0 - Configuration of Kubeflow

In [9]:
PIPELINES_HOST='1ececdcaa6ca20d7-dot-us-central2.pipelines.googleusercontent.com'
PROJECT='ind-coe'
BUCKET='cgmlops-demo'

In [10]:
import kfp
import os
client = kfp.Client(host=PIPELINES_HOST)

In [11]:
client.list_pipelines()

{'next_page_token': None,
 'pipelines': [{'created_at': datetime.datetime(2020, 9, 7, 10, 32, 32, tzinfo=tzlocal()),
                'default_version': {'code_source_url': None,
                                    'created_at': datetime.datetime(2020, 9, 7, 16, 36, 34, tzinfo=tzlocal()),
                                    'id': '82cec00e-e771-48a8-9fa9-85a777a9f268',
                                    'name': 'cgmlops_v2',
                                    'package_url': None,
                                    'parameters': [{'name': 'project',
                                                    'value': 'ind-coe'},
                                                   {'name': 'bucket',
                                                    'value': 'cgmlops-demo'},
                                                   {'name': 'start_year',
                                                    'value': '2008'}],
                                    'resource_references': [{'key': {'id': 'e

## Initial Setup - Build Docker Containers

* Process 1 - Preprocessing (BQ to CSV) 
* Process 2 - Hyper Parameter Training
* Process 3 - Training 
* Process 4 - Deploy AI Platform
* Process 5 - Deploy Application 

In [ ]:
# Build all notebooks and scripts (Step 1 - Step 5) 
%%bash
cd pipelines/containers
bash build_all.sh

In [ ]:
# Build only one containers
%%bash
cd pipelines/containers
bash build_one.sh

## Initial Setup - Kubeflow Pipeline Compilation

In [ ]:
%%bash
cd pipelines/containers
dsl-compile --py ./pipeline/mlp_babyweight.py --function preprocess_train_and_deploy --output pipeline.tar.gz

In [ ]:
pipeline_version_name="cgmlops_v2"
pipeline_version_file_path="/home/jupyter/cgmlops/pipeline.yaml"
pipeline_version_file = os.path.join(pipeline_version_file_path)
pipeline_version = client.pipeline_uploads.upload_pipeline_version(pipeline_version_file,
                                                                   name=pipeline_version_name,
                                                                   pipelineid='e656d11a-53f4-4cee-a408-d7946ca4b254')

%%bash
cd pipelines/containers
./setup_github_trigger.sh

## CICD - Execute the Experiment

In [12]:
from pipelines.containers.pipeline import mlp_babyweight

args = {
    'project' : PROJECT, 
    'bucket' : BUCKET
}

pipeline = client.create_run_from_pipeline_func(mlp_babyweight.preprocess_train_and_deploy, args)

## CICD - Scenario 2 - Modify one of the notebook script or python notebook in GitHub (bqtocsv)

## CICD - Scenario 2 - Execution of Retrain and Deploy Step alone

In [ ]:
#os.environ['HPARAM_JOB'] = 'babyweight_200207_231639' # change to job from complete step
pipeline = client.create_run_from_pipeline_func(mlp_babyweight.train_and_deploy, args)